In [169]:
"""EchoNet-Dynamic Dataset."""

import os
import collections
import pandas
import torch
import numpy as np
import skimage.draw
import torchvision
import echonet_dataloader
from echonet_dataloader.deformation.utils import *

class Echo(torchvision.datasets.VisionDataset):
    def __init__(self, root=None,
                 split="train", target_type="EF",
                 mean=0., std=1.,
                 length=16, period=2,
                 fixed_length=16, max_length=250,
                 clips=1,
                 self_supervised = False):


        super().__init__(root)
        self.root = root
        self.split = split.upper()
        if not isinstance(target_type, list):
            target_type = [target_type]
        self.target_type = target_type
        self.mean = mean
        self.std = std
        self.length = length
        self.max_length = max_length
        self.period = period
        self.clips = clips
        self.fnames, self.outcome , self.ejection  = [], [] , []
        self.fixed_length = fixed_length


        # Load video-level labels
        # Read csv file
        with open(os.path.join(self.root, "FileList.csv")) as f:
            data = pandas.read_csv(f)
            
        # Make all TRAIN,VAl and TEST upper
        data["Split"].map(lambda x: x.upper())
        
        
        # Split = Train , Val or Test
        if self.split != "ALL":
            
            data = data[data["Split"] == self.split]
            self.header = data.columns.tolist()
            self.fnames = data["FileName"].tolist()
            # File names with suffix (.avi)
            self.fnames = [fn + ".avi" for fn in self.fnames if os.path.splitext(fn)[1] == ""]  # Assume avi if no suffix
            self.outcome = data.values.tolist()

            
            
            # Check that files are present
            missing = set(self.fnames) - set(os.listdir(os.path.join(self.root, "Videos")))
            if len(missing) != 0:
                print("{} videos could not be found in {}:".format(len(missing), os.path.join(self.root, "Videos")))
                for f in sorted(missing):
                    print("\t", f)
                raise FileNotFoundError(os.path.join(self.root, "Videos", sorted(missing)[0]))

            # Load traces
            self.frames = collections.defaultdict(list)
            self.trace = collections.defaultdict(_defaultdict_of_lists)

            
            # Open VolumeTracings.csv
            with open(os.path.join(self.root, "VolumeTracings.csv")) as f:
                header = f.readline().strip().split(",")
                assert header == ["FileName", "X1", "Y1", "X2", "Y2", "Frame"]

                for line in f:
                    filename, x1, y1, x2, y2, frame = line.strip().split(',')
                    x1 = float(x1)
                    y1 = float(y1)
                    x2 = float(x2)
                    y2 = float(y2)
                    frame = int(frame)
                    
                    # New frame index for the given filename
                    if frame not in self.trace[filename]:
                        self.frames[filename].append(frame)
                        
                     # Add volume lines to trace
                    self.trace[filename][frame].append((x1, y1, x2, y2))
                    
                    
            # Changing the format to numpy array
            for filename in self.frames:
                for frame in self.frames[filename]:
                    self.trace[filename][frame] = np.array(self.trace[filename][frame])

                    
            # A small number of videos are missing traces; remove these videos
            keep = [len(self.frames[f]) >= 2 for f in self.fnames]
            
            # Prepare for getitem
            self.fnames = [f for (f, k) in zip(self.fnames, keep) if k]
            self.outcome = [f for (f, k) in zip(self.outcome, keep) if k]
            self.self_supervised = self_supervised
            self.conf = conf

    def __getitem__(self, index):
        
        # Find filename of video
        path = os.path.join(self.root, "Videos", self.fnames[index])

        # Load video into np.array
        video = echonet_dataloader.utils.loadvideo(path).astype(np.float32)


        # Scale pixel values from 0-255 to 0-1
        video /= 255.0
        # video = np.moveaxis(video, 0, 1)

        # Set number of frames
        c, f, h, w = video.shape
        
        
        # index of ED and ES 
        key = self.fnames[index]
        samp_size = abs(self.frames[key][0]-self.frames[key][-1])

        large_key = self.frames[key][-1]
        small_key = self.frames[key][0]
            
        # Index of first and last frame with segmentation
        first_poi = min(small_key, large_key)
        last_poi  = max(small_key, large_key)
        dist = abs(small_key-large_key) 
        
        # Label of frames 
        label  = np.zeros(f)
        label[small_key] = 1 # End systole (small)
        label[large_key] = 2 # End diastole (large)

            

        # Gather targets
        target = []
        for t in self.target_type:
            key = self.fnames[index]
            if t == "LargeFrame":
                target.append(video[:, self.frames[key][-1], :, :])
            elif t == "SmallFrame":
                target.append(video[:, self.frames[key][0], :, :])
            elif t in ["LargeTrace", "SmallTrace"]:
                if t == "LargeTrace":
                    t = self.trace[key][self.frames[key][-1]]
                else:
                    t = self.trace[key][self.frames[key][0]]
                x1, y1, x2, y2 = t[:, 0], t[:, 1], t[:, 2], t[:, 3]
                x = np.concatenate((x1[1:], np.flip(x2[1:])))
                y = np.concatenate((y1[1:], np.flip(y2[1:])))

                r, c = skimage.draw.polygon(np.rint(y).astype(np.int), np.rint(x).astype(np.int), (video.shape[2], video.shape[3]))
                mask = np.zeros((video.shape[2], video.shape[3]), np.float32)
                mask[r, c] = 1
                target.append(torch.tensor(mask))
            else:
                if self.split == "CLINICAL_TEST" or self.split == "EXTERNAL_TEST":
                    target.append(torch.tensor(np.float32(0)))
                else:
                    target.append(torch.tensor(np.float32(self.outcome[index][self.header.index(t)])))


        # Gather Video
        if dist<length:
            # Take random clips from video
            pre_start = int((length - dist)//2)
            start_index = int(max(0, first_poi - pre_start))  
            end_index = start_index + length 
            video = video[:,start_index : end_index, :, :]
            label = label[start_index:end_index]
        
        else:
            divider     = np.random.random_sample()*5+2
            start_index = first_poi - dist//divider
            start_index = int(max(0, start_index)//2*2)             
            divider     = np.random.random_sample()*5+2
            end_index   = last_poi +1 + dist//divider #+1 to INCLUDE the frame
            end_index   = int(min(f, end_index)//2*2)
            step = int( np.ceil((end_index-start_index)/ length) )
            list_frame = np.arange(first_poi, last_poi , step, dtype=int)
            list_frame = np.append(list_frame, last_poi)
            list_all = np.arange(start_index,end_index)
            np.random.shuffle(list_all)
            list_frame  = np.sort(np.append(list_frame,list_all[:(length - len(list_frame))]))
            video = video[:,list_frame, :, :]
            label = label[list_frame]
                
        if self.self_supervised:
            deform = generate_pair_d(video,1, self.conf)
            #cine_d = torch.as_tensor(np.array(deform).astype('float'))
            return video,deform,label,target
            
        return video,label, target

    def __len__(self):
        return len(self.fnames)

    def extra_repr(self) -> str:
        """Additional information to add at end of __repr__."""
        lines = ["Target type: {target_type}", "Split: {split}"]
        return '\n'.join(lines).format(**self.__dict__)


def _defaultdict_of_lists():
    """Returns a defaultdict of lists.

    This is used to avoid issues with Windows (if this function is anonymous,
    the Echo dataset cannot be used in a dataloader).
    """

    return collections.defaultdict(list)

In [165]:
root = '/AS_Neda/echonet/'
with open(os.path.join(root, "FileList.csv")) as f:
    data = pandas.read_csv(f)
data

,Unnamed: 0,FileName,EF,ESV,EDV,FrameHeight,FrameWidth,FPS,NumberOfFrames,Split,EDFrame,ESFrame
0,0,0X100009310A3BD7FC,78.498406,14.881368,69.210534,112,112,50,174,VAL,46.0,61.0
1,1,0X1002E8FBACD08477,59.101988,40.383876,98.742884,112,112,50,215,TRAIN,3.0,18.0
2,2,0X1005D03EED19C65B,62.363798,14.267784,37.909734,112,112,50,104,TRAIN,24.0,35.0
3,3,0X10075961BC11C88E,54.545097,33.143084,72.914210,112,112,55,122,TRAIN,91.0,108.0
4,4,0X10094BA0A028EAC3,24.887742,127.581945,169.855024,112,112,52,207,VAL,137.0,156.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10025,10025,0X234005774F4CB5CD,51.724743,47.065329,97.493690,768,1040,50,127,TRAIN,NaN,NaN
10026,10026,0X2DC68261CBCC04AE,62.187781,26.333478,69.642772,768,1024,50,66,TRAIN,NaN,NaN
10027,10027,0X35291BE9AB90FB89,62.070762,49.064338,129.357561,768,1024,50,208,TRAIN,NaN,NaN
10028,10028,0X6C435C1B417FDE8A,59.635257,57.721170,142.998978,768,1024,50,166,TRAIN,NaN,NaN


In [170]:
import echonet_dataloader
from echonet_dataloader import utils
from config import models_genesis_config
from get_config_genesis import get_config
import os
from config import models_genesis_config

os.environ["CUDA_VISIBLE_DEVICES"] = "1,7"

conf = models_genesis_config()
config = get_config()
tasks = ['EF','SmallFrame' , 'LargeFrame', 'SmallTrace' ,'LargeTrace']
kwargs = {"target_type": tasks,
          "mean": 0,
          "std": 1,
          "self_supervised":True
          }
traindataset = Echo(root='/AS_Neda/echonet/', split="train",**kwargs)
#train_loader = torch.utils.data.DataLoader(
#                    traindataset, batch_size=16, num_workers=4, shuffle=True, pin_memory=True, drop_last=True)
#video,target = next(iter(train_loader))

In [171]:
video,deform,label,target = traindataset[750]
deform.shape

(3, 16, 112, 112)

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter

fig, ax = plt.subplots()
video,deform,label,target = traindataset[750]
print('label',label)
video = (np.array(deform)).transpose((1,2,3,0))
frames = [[ax.imshow(video[i],cmap='gray')] for i in range(len(video))]
ani = animation.ArtistAnimation(fig, frames)
ani

In [ ]:
# Load trained segmentation model
model_s = torchvision.models.segmentation.deeplabv3_resnet50(aux_loss=False)
model_s.classifier[-1] = torch.nn.Conv2d(model_s.classifier[-1].in_channels, 1, kernel_size=model_s.classifier[-1].kernel_size)
if device.type == "cuda":
    model_s = torch.nn.DataParallel(model_s)
model_s.to(device)